## Postprocessing randomized measurement on IBM quantum computers (

This tutorial illustrates how to acquire and save randomized measurements on a quantum computer using Qiskit (RandomizedMeasurementsQiskit.ipynb). This data can be then postprocessed using our julia library RandomMeas (RandomizedMeasurementsQiskitPostprocessing.ipynb)

In [ ]:
using RandomMeas
using NPZ

### Loading the data and retrieving the parameters of the experiment

In [2]:
job_id = "6f33bc3a-1479-4127-918e-5c43b9d407e4"
local_unitary_ = npzread("data/"*job_id*"_u.npy")
data_cal_ = 2 .- npzread("data/"*job_id*"_data.npy")[1,:,:,:]; #Data obtained from the circuit qc_meas used for calibrating robust shadows
data_ = 2 .- npzread("data/"*job_id*"_data.npy")[2,:,:,:]; #Data obtained from the preparation of the cluster state
NU,NM,N = size(data_)
@show NU,NM,N;

SystemError: SystemError: opening file "data/6f33bc3a-1479-4127-918e-5c43b9d407e4_u.npy": No such file or directory

### Converting the data into RandomMeas.jl format 

In [3]:
ξ = siteinds("Qubit",N)
data_cal = Vector{MeasurementData{LocalUnitaryMeasurementSetting}}(undef,NU)
data = Vector{MeasurementData{LocalUnitaryMeasurementSetting}}(undef,NU)

for r in 1:NU
    local_unitary = Vector{ITensor}()
    for i in 1:N
        push!(local_unitary,ITensor(local_unitary_[r,i,:,:],ξ[i]',ξ[i]))
    end
    measurement_setting = LocalUnitaryMeasurementSetting(N,local_unitary,ξ)
    data_cal[r] = MeasurementData(data_cal_[r,:,:];measurement_setting=measurement_setting)
    data[r] = MeasurementData(data_[r,:,:];measurement_setting=measurement_setting)
end
group_cal = MeasurementGroup(data_cal)
group = MeasurementGroup(data);

UndefVarError: UndefVarError: `N` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Extraction of the calibration parameters $G$ for building robust shadows

For purely readout error models, we have $G\approx 1-p$ where $p$ is the readout error 

In [4]:
states = ["Dn" for n in 1:N]
ψ0  = MPS(ComplexF64,ξ,states);
G_e = get_calibration_vector(ψ0,group_cal)
print("Calibrated RM parameters: ", G_e)

UndefVarError: UndefVarError: `N` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Extracting the purity

We extract the purity of the first two qubits, and the purity of the four qubits. As expected for a cluster state, we have $p(12)\approx 0.5$, and $p(1234)\approx 1$.  

In [5]:
ρ = get_dense_shadows(group,number_of_ru_batches=10)
ρ_sub = partial_trace(ρ,collect(1:2))
ρ_mitigated = get_dense_shadows(group,number_of_ru_batches=10;G=G_e)
ρ_mitigated_sub = partial_trace(ρ_mitigated,collect(1:2))
println("SubSystem Purity (unmitigated) ", get_trace_moment(ρ_sub,2))
println("SubSystem Purity (mitigated) ", get_trace_moment(ρ_mitigated_sub,2))
println("Total Purity (unmitigatd) ", get_trace_moment(ρ,2))
println("Total Purity (mitigated) ", get_trace_moment(ρ_mitigated,2))

UndefVarError: UndefVarError: `group` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing SplitApplyCombine in the current active module Main